In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical

In [37]:
path="/Users/ugur/Desktop/UrbansoundProject/UrbanSound8K/"
metadata=pd.read_csv(path+"metadata/UrbanSound8K.csv")
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [38]:
fold_number=np.array(metadata["fold"])
file_name=np.array(metadata["slice_file_name"])
class_name=np.array(metadata["class"])
classID=metadata["classID"]

In [39]:
def resized_dim(img,rate):
    #height = img.shape[0]
    #width = img.shape[1]
    #channels = img.shape[2]
    width=int(img.shape[1])
    height=int(img.shape[0])
    if width>=height:
        width=height*rate/100
        height=height*rate/100
    else:
        height=width*rate/100
        width=width*rate/100
    return int(height),int(width)

In [40]:
test_im = cv2.imread('test.png')
new_shape=resized_dim(test_im,13.5)
print(new_shape)
print(new_shape[0]**2)

(64, 64)
4096


In [57]:
audio_path=path+"audio"
Vision=[]
Id=[]
for i in range(len(classID)):
    file_path=audio_path + "/fold" +str(fold_number[i]) + "/" +file_name[i]
    image_path=file_path[:-3]+"png"
    img=cv2.imread(image_path)
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img=cv2.resize(gray_img,new_shape, interpolation = cv2.INTER_AREA)
    normalized_img=resized_img/255
    Vision.append(normalized_img)
    Id.append(classID[i])

df=pd.DataFrame({"Vision":Vision,"ClassID":classID})
df.head()

,Vision,ClassID
0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",3
1,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",2
2,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",2
3,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",2
4,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",2


In [58]:
print(len(df["Vision"]))
print(df["Vision"][100].shape)
print(df["Vision"][100])

8732
(64, 64)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [62]:
print(df["ClassID"])
print(df["ClassID"].unique())

0       3
1       2
2       2
3       2
4       2
       ..
8727    1
8728    1
8729    1
8730    1
8731    1
Name: ClassID, Length: 8732, dtype: int64
[3 2 1 0 9 6 8 5 7 4]


In [69]:
len(df["ClassID"].unique())

10

In [70]:
df["ClassID"]=train_labels = to_categorical(df["ClassID"],num_classes=len(df["ClassID"].unique()))

In [74]:
len(df["ClassID"])

8732

In [73]:
df.ClassID.unique()

array([0., 1.], dtype=float32)

In [32]:
x_train, x_test, y_train, y_test = train_test_split(df["Vision"], df["ClassID"], test_size=0.2, random_state = 42)
x_val,x_test,y_val,y_test=train_test_split(x_test,y_test,train_size=0.5,random_state=42)

In [33]:
print(f"Length of Xtrain: {len(x_train)}")
print(f"Length of Xtest: {len(x_test)}")
print(f"Length of Xvalidation: {len(x_val)}")
print(f"Length of Ytrain: {len(y_train)}")
print(f"Length of Ytest: {len(y_test)}")
print(f"Length of Yvalidation: {len(y_val)}")

Length of Xtrain: 6985
Length of Xtest: 874
Length of Xvalidation: 873
Length of Ytrain: 6985
Length of Ytest: 874
Length of Yvalidation: 873


In [34]:
x_val.to_pickle("x_validation")
x_train.to_pickle("x_train")
x_test.to_pickle("x_test")
y_val.to_pickle("y_validation")
y_train.to_pickle("y_train")
y_test.to_pickle("y_test")